# sequel

In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import random

def load_dataset(path="./datas/hetrec2011-lastfm-2k"):
    # Load user-artist interactions
    user_artists = pd.read_csv(f"{path}/user_artists.dat", sep='\t')

    # Load artists data
    artists = pd.read_csv(f"{path}/artists.dat", sep='\t')

    return user_artists, artists

# Preprocess data
def preprocess_data(user_artists):
    # Create user-item matrix
    user_item_matrix_df = user_artists.pivot(index='userID', columns='artistID', values='weight').fillna(0)
    return user_item_matrix_df

# userCF核心原理

In [24]:
def user_based_cf(user_item_matrix_df, user_id, n_users=10, n_recommendations=10):
    # Calculate user similarity
    user_similarity = cosine_similarity(user_item_matrix_df)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix_df.index, columns=user_item_matrix_df.index)

    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:n_users+1].index

    # Get recommendations
    recommendations = defaultdict(float)

    # 对于当前没听过的艺术家，以相似用户喜欢的艺术家频次为准,
    # recommendations[item] = 累加（某user和其相似用户的相似度 * weight）
    current_user_mean_weight = user_item_matrix_df.loc[user_id].mean()

    for similar_user in similar_users:
        similarity_between_user = user_similarity_df.loc[user_id, similar_user]
        similar_user_mean_weight = user_item_matrix_df.loc[similar_user].mean()

        for item in user_item_matrix_df.columns:
            if user_item_matrix_df.loc[user_id, item] == 0 and user_item_matrix_df.loc[similar_user, item] > 0:
                recommendations[item] += similarity_between_user * (user_item_matrix_df.loc[similar_user, item] - similar_user_mean_weight)

    recommendations = {item: score + current_user_mean_weight for item, score in recommendations.items()}
    # Sort recommendations
    recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]
    return recommendations

# itemCf核心原理

In [25]:
def item_based_cf(user_item_matrix_df, user_id, n_items=10, n_recommendations=10):
    # 1. Calculate item similarity between items
    item_similarity = cosine_similarity(user_item_matrix_df.T)
    item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix_df.columns, columns=user_item_matrix_df.columns)

    # 2. Get items the user has interacted with and weights > 0
    user_related_items = user_item_matrix_df.loc[user_id]
    related_items_id = user_related_items[user_related_items > 0].index.tolist()

    # Get recommendations
    recommendations = defaultdict(float)

    for item_id in related_items_id:
        item_weight = user_item_matrix_df.loc[user_id, item_id]

        # 2.1 Find n top items similar to current item, according to item_similarity_df
        similar_items = item_similarity_df[item_id].sort_values(ascending=False)[1:n_items+1]

        for similar_item, similarity_between_item in similar_items.items():
            # 2.2 only find the similar item is not in related_items_id, add it to recommendations
            if similar_item in related_items_id:
                continue
            # the below similar_item's weight must be 0
            recommendations[similar_item] += similarity_between_item * item_weight

    # Sort recommendations
    recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]
    return recommendations

In [31]:
def is_sparse_matrix(df, threshold=0.1):
    """
    判断一个DataFrame是否是稀疏矩阵

    参数:
    df (pd.DataFrame): 需要判断的DataFrame
    threshold (float): 稀疏矩阵的阈值，非零元素比例低于该值则认为是稀疏矩阵

    返回:
    bool: 如果是稀疏矩阵返回True，否则返回False
    """
    total_elements = df.size
    non_zero_elements = df.astype(bool).sum().sum()
    sparsity_ratio = non_zero_elements / total_elements
    return sparsity_ratio

# main运行

In [32]:
def main():
    # Load and preprocess data
    user_artists, artists = load_dataset()
    user_item_matrix_df = preprocess_data(user_artists)
    print("是否是稀疏矩阵:", is_sparse_matrix(user_item_matrix_df))
    # Example: Get recommendations for a specific user
    user_id = user_item_matrix_df.index[0]  # First user in the dataset
    # user_id = int(input("input the user id:"))

    print(f"User-based recommendations for user {user_id}:")
    user_recommendations = user_based_cf(user_item_matrix_df, user_id)
    for item_id, score in user_recommendations:
        artist_name = artists[artists['id'] == item_id]['name'].values[0] if item_id in artists['id'].values else "Unknown"
        print(f"Artist ID: {item_id}, Score: {score:.2f}, Name: {artist_name}")

    print(f"\nItem-based recommendations for user {user_id}:")
    item_recommendations = item_based_cf(user_item_matrix_df, user_id)
    for item_id, score in item_recommendations:
        artist_name = artists[artists['id'] == item_id]['name'].values[0] if item_id in artists['id'].values else "Unknown"
        print(f"Artist ID: {item_id}, Score: {score:.2f}, Name: {artist_name}")

main()

是否是稀疏矩阵: 0.002782815119924182
User-based recommendations for user 2:
Artist ID: 511, Score: 11031.56, Name: U2
Artist ID: 159, Score: 9437.26, Name: The Cure
Artist ID: 1001, Score: 8636.79, Name: Pet Shop Boys
Artist ID: 2562, Score: 6256.60, Name: Arcadia
Artist ID: 1014, Score: 5248.06, Name: Erasure
Artist ID: 993, Score: 5025.80, Name: Simple Minds
Artist ID: 187, Score: 4850.40, Name: a-ha
Artist ID: 4313, Score: 4472.78, Name: Nephew
Artist ID: 227, Score: 3799.39, Name: The Beatles
Artist ID: 6776, Score: 3573.63, Name: Book of Love

Item-based recommendations for user 2:
Artist ID: 2556, Score: 14102.14, Name: The Power Station
Artist ID: 8995, Score: 13904.49, Name: Andy Taylor
Artist ID: 1076, Score: 13796.74, Name: Wham!
Artist ID: 2562, Score: 13181.35, Name: Arcadia
Artist ID: 13161, Score: 12606.11, Name: Private
Artist ID: 4313, Score: 12600.67, Name: Nephew
Artist ID: 6350, Score: 12455.23, Name: TV-2
Artist ID: 996, Score: 11801.53, Name: Mike & The Mechanics
Artist I